<a href="https://colab.research.google.com/github/chefinha-tech/acad_control/blob/main/verificador_certificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import ssl
import socket
import datetime

def verificar_certificado_site(hostname, port=443):
    """
    Verifica a validade do certificado SSL/TLS de um site.
    Args:
        hostname (str): O nome do host (domínio) do site.
        port (int): A porta (padrão 443 para HTTPS).
    Returns:
        dict: Um dicionário com o status do certificado e informações.
    """
    context = ssl.create_default_context()
    try:
        with socket.create_connection((hostname, port)) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()

                # Extrair datas de validade
                not_before_str = cert['notBefore']
                not_after_str = cert['notAfter']

                # Converter para objetos datetime e garantir que são UTC
                try:
                    # O formato '%Z' em strptime pode não ser robusto para todos os sistemas/locales.
                    # Para 'GMT', ele geralmente funciona, mas vamos garantir o UTC de forma explícita.
                    # Uma abordagem mais robusta para parsear datas com fuso horário seria usar dateutil.parser
                    # ou tratar o GMT como UTC.
                    # Vamos tentar parsear sem o %Z e adicionar o UTC se a string terminar em 'GMT'.
                    if not_before_str.endswith(' GMT'):
                        not_before_dt_naive = datetime.datetime.strptime(not_before_str[:-4], '%b %d %H:%M:%S %Y')
                        not_before = not_before_dt_naive.replace(tzinfo=datetime.timezone.utc)
                    else:
                        not_before = datetime.datetime.strptime(not_before_str, '%b %d %H:%M:%S %Y %Z') # Fallback, mas pode falhar
                        if not_before.tzinfo is None: # Se ainda for naive, assumir UTC
                            not_before = not_before.replace(tzinfo=datetime.timezone.utc)


                    if not_after_str.endswith(' GMT'):
                        not_after_dt_naive = datetime.datetime.strptime(not_after_str[:-4], '%b %d %H:%M:%S %Y')
                        not_after = not_after_dt_naive.replace(tzinfo=datetime.timezone.utc)
                    else:
                        not_after = datetime.datetime.strptime(not_after_str, '%b %d %H:%M:%S %Y %Z') # Fallback, mas pode falhar
                        if not_after.tzinfo is None: # Se ainda for naive, assumir UTC
                            not_after = not_after.replace(tzinfo=datetime.timezone.utc)

                except ValueError:
                    return {
                        "status": "Erro na data",
                        "hostname": hostname,
                        "detalhes": "Não foi possível parsear as datas de validade do certificado. Verifique o formato."
                    }

                # Obter o tempo atual em UTC
                current_time = datetime.datetime.now(datetime.timezone.utc)

                status = "Válido"
                if current_time < not_before:
                    status = "Ainda não válido (data futura)"
                elif current_time > not_after:
                    status = "Expirado"

                return {
                    "status": status,
                    "hostname": hostname,
                    "not_before": not_before.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "not_after": not_after.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "dias_restantes": (not_after - current_time).days if status == "Válido" else None,
                    "issuer": dict(x[0] for x in cert['issuer']) # Informações do emissor
                }

    except ssl.SSLCertVerificationError as e:
        return {
            "status": "Erro de verificação SSL",
            "hostname": hostname,
            "detalhes": str(e)
        }
    except socket.gaierror:
        return {
            "status": "Erro de DNS",
            "hostname": hostname,
            "detalhes": "Não foi possível resolver o nome do host."
        }
    except ConnectionRefusedError:
        return {
            "status": "Conexão recusada",
            "hostname": hostname,
            "detalhes": "A conexão foi recusada pelo servidor. Porta incorreta ou firewall?"
        }
    except Exception as e:
        return {
            "status": "Erro inesperado",
            "hostname": hostname,
            "detalhes": str(e)
        }

# --- Como usar a automação ---
if __name__ == "__main__":
    sites_para_verificar = [
        "google.com",
        "certifica.com.br",
        "globalsign.com",
        "digicert.com",
        "expired.badssl.com", # Exemplo de site com certificado expirado
        "google.com:80", # Porta incorreta para HTTPS
        "sitequenaoexiste12345.com" # Exemplo de site que não existe
    ]

    print("Iniciando verificação de certificados SSL/TLS...\n")
    for site in sites_para_verificar:
        print(f"Verificando: {site}")
        resultado = verificar_certificado_site(site)
        for key, value in resultado.items():
            print(f"  {key}: {value}")
        print("-" * 30)

Iniciando verificação de certificados SSL/TLS...

Verificando: google.com
  status: Válido
  hostname: google.com
  not_before: 2025-06-02 08:35:30 UTC
  not_after: 2025-08-25 08:35:29 UTC
  dias_restantes: 60
  issuer: {'countryName': 'US', 'organizationName': 'Google Trust Services', 'commonName': 'WR2'}
------------------------------
Verificando: certifica.com.br
  status: Válido
  hostname: certifica.com.br
  not_before: 2025-02-07 00:00:00 UTC
  not_after: 2026-03-08 23:59:59 UTC
  dias_restantes: 256
  issuer: {'countryName': 'US', 'organizationName': 'DigiCert Inc', 'organizationalUnitName': 'www.digicert.com', 'commonName': 'RapidSSL TLS RSA CA G1'}
------------------------------
Verificando: globalsign.com
  status: Válido
  hostname: globalsign.com
  not_before: 2024-10-07 04:12:02 UTC
  not_after: 2025-11-06 14:31:01 UTC
  dias_restantes: 134
  issuer: {'countryName': 'BE', 'organizationName': 'GlobalSign nv-sa', 'commonName': 'GlobalSign Extended Validation CA - SHA256 - G3